In [ ]:
#|default_exp sync

# sync

> Propagating small changes in the library back to notebooks

The library is primarily developed in notebooks so any big changes should be made there. But sometimes, it's easier to fix small bugs or typos in the modules directly. `nbprocess_update_lib` is the function that will propagate those changes back to the corresponding notebooks. Note that you can't create new cells or reorder cells with that functionality, so your corrections should remain limited.

In [ ]:
#|export
from nbprocess.imports import *
from nbprocess.read import *
from nbprocess.maker import *
from nbprocess.process import *
from nbprocess.export import *

from execnb.nbio import *
from fastcore.script import *
from fastcore.xtras import *

import ast,tempfile,json

In [ ]:
#|hide
from pdb import set_trace

In [ ]:
#|export
def absolute_import(name, fname, level):
    "Unwarps a relative import in `name` according to `fname`"
    if not level: return name
    mods = fname.split(os.path.sep)
    if not name: return '.'.join(mods)
    return '.'.join(mods[:len(mods)-level+1]) + f".{name}"

In [ ]:
test_eq(absolute_import('xyz', 'nbprocess', 0), 'xyz')
test_eq(absolute_import('', 'nbprocess', 1), 'nbprocess')
test_eq(absolute_import('core', 'nbprocess', 1), 'nbprocess.core')
test_eq(absolute_import('core', 'nbprocess/vision', 2), 'nbprocess.core')
test_eq(absolute_import('transform', 'nbprocess/vision', 1), 'nbprocess.vision.transform')
test_eq(absolute_import('notebook.core', 'nbprocess/data', 2), 'nbprocess.notebook.core')

In [ ]:
#|export
_re_import = re.compile("from\s+\S+\s+import\s+\S")

In [ ]:
#|hide
assert _re_import.match('from foo import bar')
assert not _re_import.match('#from foo import bar')

In [ ]:
#|export
def _to_absolute(code, lib_name):
    if not _re_import.search(code): return code
    res = update_import(code, ast.parse(code).body, lib_name, absolute_import)
    return ''.join(res) if res else code

def _update_lib(nbname, nb_locs, lib_name=None):
    if lib_name is None: lib_name = get_config().lib_name
    nbp = NBProcessor(nbname, ExportModuleProc(), rm_directives=False)
    nb = nbp.nb
    nbp.process()

    for name,idx,code in nb_locs:
        assert name==nbname
        cell = nb.cells[int(idx)-1]
        lines = cell.source.splitlines(True)
        directives = ''.join(cell.source.splitlines(True)[:len(cell.directives_)])
        cell.source = directives + _to_absolute(code, lib_name)
    write_nb(nb, nbname)

def _get_call(s):
    top,*rest = s.splitlines()
    return *top.split(),'\n'.join(rest)

In [ ]:
#|export
@call_parse
def nbprocess_update(fname:str): # A python file name to convert
    "Propagates any change in the modules matching `fname` to the notebooks that created them"
    if os.environ.get('IN_TEST',0): return
    code_cells = Path(fname).read_text().split("\n# %% ")[1:]
    locs = L(_get_call(s) for s in code_cells if not s.startswith('auto '))
    for nbname,nb_locs in groupby(locs, 0).items(): _update_lib(nbname, nb_locs)

In [ ]:
#|hide
# nbprocess_update("../nbprocess/sync.py")

## Export -

In [ ]:
#|hide
#|eval: false
from nbprocess.doclinks import nbprocess_export
nbprocess_export()